<a href="https://colab.research.google.com/github/hsreekumar/Data_capstone/blob/main/Simplest_NLP_Tutorial(Random_Forest_%26_Naive_Bayes).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'stocknews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F129%2F792900%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240728%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240728T193814Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D24666fe8a099752eee2d3dacbec32f4c993587765f94423f4888c6b29a0d5babb49e1e66a777515ca87df47696b110fe737110573612f4251511deac467c317fdb28a2347310a12aa88579ca5c62984043a7bab0978b69e1a0d044a1d743bc887085500e5a539d3b49dce7f1cde220f10172f65f0c8aa736f0dd45a3b448790462e853163dee2e49a82d54a8cdaea1bc409c30f46328911eef310b3a7b7c2037174b192086decd20c86e2d573bded856f39ff757e65b9af0281de7df93ee26d785ce8a33d01128c6dbbe233b0d601940967acc769db9b2ec0da12edcdf862145a8daed1f9a2e76c32b42898c064d07da57759d3527f6e9cb2419d3c309ba367e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 6097730 bytes downloaded
Downloaded and uncompressed: stocknews
Data source import complete.


# Stock Sentiment Analysis using news headlines.

# Data set

Daily News for Stock Market Prediction
https://www.kaggle.com/datasets/aaron7sun/stocknews

We will predict whether the stock price for a given stock will raise or drop based on the related news headlines.

# Topic Covered

Classifiers: Random Forest vs Naive Bayes

Feature Transformation: Count Vectorizer vs TF-IDF Vectorizer

![](https://media-exp1.licdn.com/dms/image/C5112AQFKtWwV4Gd2Lg/article-cover_image-shrink_600_2000/0/1549570474712?e=1654128000&v=beta&t=jPyk2pQW6L1bxbYJlpzve_BuRwGjoOUhxl5GUDK8Eac)


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

#Encoding are in general UTF-8(default setting), Latin-1 (also known as ISO-8859-1) or Windows-1251
df=pd.read_csv('../input/stocknews/Combined_News_DJIA.csv', encoding = "ISO-8859-1")


# Data Inspection
First, let's have a look at our data set...
It seems our data is made of different news headlines related to a company and a label of wether the companies'stock price will increase or decrease.(0:decrease, 1: increase)

In [3]:
df.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

# Train-Test Split

In time series problems, old data are used to predict later data. Therefore, we set test data to be the latest data.

In [4]:
train = df[df['Date'] < '20150101']
test = df[df['Date'] > '20141231']

# Feature Transformation
From inpecting the data, we learned that there are a lot of punctuations which will not contribute to understanding the sentiment in the message. We can use "replace("[^a-zA-Z]"," ",regex=True, inplace=True)" to replace everything apart from a-z and A-Z with blank space and store in-place in the "data" variable.

In other NLP applications, stop words like articles, prepositions, pronouns, conjunction are removed to give more emphasis to the important words. It was not done here as news headlines are generally condensed and every word have particular meaning.

Regular expressions (regex) are essentially text patterns that you can use to automate searching through and replacing elements within strings of text. This can make cleaning and working with text-based data sets much easier, saving you the trouble of having to search through mountains of text by hand.

In [5]:
# Removing punctuations
data=train.iloc[:,2:27]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(25)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
data.head(5)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,b Georgia downs two Russian warplanes as cou...,b BREAKING Musharraf to be impeached,b Russia Today Columns of troops roll into So...,b Russian tanks are moving towards the capital...,b Afghan children raped with impunity U N ...,b Russian tanks have entered South Ossetia...,b Breaking Georgia invades South Ossetia Rus...,b The enemy combatent trials are nothing but...,b Georgian troops retreat from S Osettain cap...,b Did the U S Prep Georgia for War with Russia,...,b Georgia Invades South Ossetia if Russia ge...,b Al Qaeda Faces Islamist Backlash,b Condoleezza Rice The US would not act to p...,b This is a busy day The European Union has ...,b Georgia will withdraw soldiers from Ir...,b Why the Pentagon Thinks Attacking Iran is a ...,b Caucasus in crisis Georgia invades South Os...,b Indian shoe manufactory And again in a se...,b Visitors Suffering from Mental Illnesses Ban...,b No Help for Mexico s Kidnapping Surge
1,b Why wont America and Nato help us If they w...,b Bush puts foot down on Georgian conflict,b Jewish Georgian minister Thanks to Israeli ...,b Georgian army flees in disarray as Russians ...,b Olympic opening ceremony fireworks faked,b What were the Mossad with fraudulent New Zea...,b Russia angered by Israeli military sale to G...,b An American citizen living in S Ossetia blam...,b Welcome To World War IV Now In High Definit...,b Georgia s move a mistake of monumental prop...,...,b Israel and the US behind the Georgian aggres...,b Do not believe TV neither Russian nor Geor...,b Riots are still going on in Montreal Canada...,b China to overtake US as largest manufacturer,b War in South Ossetia PICS,b Israeli Physicians Group Condemns State Tort...,b Russia has just beaten the United States ov...,b Perhaps the question about the Georgia R...,b Russia is so much better at war,b So this is what it s come to trading sex fo...
2,b Remember that adorable year old who sang a...,b Russia ends Georgia operation,b If we had no sexual harassment we would hav...,b Al Qa eda is losing support in Iraq because ...,b Ceasefire in Georgia Putin Outmaneuvers the...,b Why Microsoft and Intel tried to kill the XO...,b Stratfor The Russo Georgian War and the Bal...,b I m Trying to Get a Sense of This Whole Geor...,b The US military was surprised by the timing ...,b U S Beats War Drum as Iran Dumps the Dollar,...,b U S troops still in Georgia did you know t...,b Why Russias response to Georgia was right,b Gorbachev accuses U S of making a serious ...,b Russia Georgia and NATO Cold War Two,b Remember that adorable year old who led y...,b War in Georgia The Israeli connection,b All signs point to the US encouraging Georgi...,b Christopher King argues that the US and NATO...,b America The New Mexico,b BBC NEWS Asia Pacific Extinction by man...
3,b U S refuses Israel weapons to attack Iran ...,b When the president ordered to attack Tskhinv...,b Israel clears troops who killed Reuters cam...,b Britain s policy of being tough on drugs is...,b Body of year old found in trunk Latest ...,b China has moved million quake survivors...,b Bush announces Operation Get All Up In Russi...,b Russian forces sink Georgian ships,b The commander of a Navy air reconnaissance s...,b of CNN readers Russia s actions in Geor...,...,b Elephants extinct by,b US humanitarian missions soon in Georgia i...,b Georgia s DDOS came from US sources,b Russian convoy heads into Georgia violating...,b Israeli defence minister US against strike ...,b Gorbachev We Had No Choice,b Witness Russian forces head towards Tbilisi...,b Quarter of Russians blame U S for conflict...,b Georgian president says US military will ta...,b Nobel laureate Aleksander Solzhenitsyn...
4,b All the experts admit that we should legalis...,b War in South Osetia pictures made by a ...,b Swedish wrestler Ara Abrahamian throws away ...,b Russia exaggerated the death toll in South O...,b Missile That Killed Inside Pakistan May Ha...,b Rushdie Cond

As the case of letters in the words does not affect its sentiment, we can convert all the characters to lower case.

In [6]:
# Convertng headlines to lower case
for index in new_Index:
    data[index]=data[index].str.lower()
data.head(1)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,b georgia downs two russian warplanes as cou...,b breaking musharraf to be impeached,b russia today columns of troops roll into so...,b russian tanks are moving towards the capital...,b afghan children raped with impunity u n ...,b russian tanks have entered south ossetia...,b breaking georgia invades south ossetia rus...,b the enemy combatent trials are nothing but...,b georgian troops retreat from s osettain cap...,b did the u s prep georgia for war with russia,...,b georgia invades south ossetia if russia ge...,b al qaeda faces islamist backlash,b condoleezza rice the us would not act to p...,b this is a busy day the european union has ...,b georgia will withdraw soldiers from ir...,b why the pentagon thinks attacking iran is a ...,b caucasus in crisis georgia invades south os...,b indian shoe manufactory and again in a se...,b visitors suffering from mental illnesses ban...,b no help for mexico s kidnapping surge


Next, we are going to combine the headings so that we can convert it to a vector so that we can apply the it to the NLP (NLP models usually involve working with vectors).

We will join every headings a link them with blank spaces. ".iloc" is Purely integer-location based indexing for selection by position.

In [7]:
headlines = []
for row in range(0,len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row,0:25]))

All the sentences are included in a list as shown below.

In [8]:
headlines[0]

'b georgia  downs two russian warplanes  as countries move to brink of war  b breaking  musharraf to be impeached   b russia today  columns of troops roll into south ossetia  footage from fighting  youtube   b russian tanks are moving towards the capital of south ossetia  which has reportedly been completely destroyed by georgian artillery fire  b afghan children raped with  impunity   u n  official says   this is sick  a three year old was raped and they do nothing  b     russian tanks have entered south ossetia whilst georgia shoots down two russian jets   b breaking  georgia invades south ossetia  russia warned it would intervene on so s side  b the  enemy combatent  trials are nothing but a sham  salim haman has been sentenced to       years  but will be kept longer anyway just because they feel like it   b georgian troops retreat from s  osettain capital  presumably leaving several hundred people killed   video   b did the u s  prep georgia for war with russia   b rice gives green

# Count Vectorizer
Count Vectorizer will represent words by numbers so that they can be processed by algorithms. This is done by making each unique word a column and count the number of times each word appear and then make it the row values. An example is given below.

![](https://www.educative.io/api/edpresso/shot/5197621598617600/image/6596233398321152)

ngram is set as 2,2 so that pairs of words appearing together are used as features (columns)
If we set (1, 1) means only unigrams(sngle words), (1, 2) means unigrams and bigrams(2 words), and (2, 2) means only bigrams.

In [9]:
## implement BAG OF WORDS
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

# Model - RandomForest
RandomForest is a flexible, easy to use machine learning algorithm that produces, even without hyper-parameter tuning, a great result most of the time. It is also one of the most used algorithms, because of its simplicity and diversity. It works by creating a set of decision trees from a randomly selected subset of the training set. It is basically a set of decision trees (DT) from a randomly selected subset of the training set and then It collects the votes from different decision trees to decide the final prediction.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/76/Random_forest_diagram_complete.png/220px-Random_forest_diagram_complete.png)

The parameters for our classifier are as follow:
* n_estimators : This is the number of trees you want to build before taking the maximum voting or averages of predictions. Higher number of trees give you better performance but makes your code slower.
* crierion : This is how each node of the tree is selected. There are 2 options:
  * gini: much faster
  * entropy: obtained results are slightly better

RandomForest Classifier is used here because we want sboth speed and accuracy.

For details on selecting ML models, please refer to the below cheatsheet.
![](https://miro.medium.com/max/1400/1*fXm06pz5UGX_N_Y0j9X2IA.png)

In [10]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

Same feature transformations are done for the test set:


In [11]:
## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = countvector.transform(test_transform)
predictions = randomclassifier.predict(test_dataset)

Great! We have completed oour first prediction. Let's see how it performed.

# Evaluation for Random Forest + Count Vectorizer

First we will take a look at its confusion matrix

![](http://rasbt.github.io/mlxtend/user_guide/evaluate/confusion_matrix_files/confusion_matrix_1.png)

Then let's take a look at the accuracy score and a detailed report.

Support is the number of actual occurrences of the class in the specified datase. We can see if the samples are balanced or not.

The other terms related are defined as:

![](https://i.ytimg.com/vi/ji48Lz6amMc/maxresdefault.jpg)



In [12]:
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

[[132  54]
 [  0 192]]
0.8571428571428571
              precision    recall  f1-score   support

           0       1.00      0.71      0.83       186
           1       0.78      1.00      0.88       192

    accuracy                           0.86       378
   macro avg       0.89      0.85      0.85       378
weighted avg       0.89      0.86      0.85       378



From the similar supports across two classes, we see that the dataset is balanced.
The Accuracy is decent as well.

In CountVectorizer we only count the number of times a word appears in the document which results in biasing in favour of most frequent words. this ends up in ignoring rare words which could have helped is in processing our data more efficiently.

To overcome this , we use TfidfVectorizer .

# TF-IDF
TF-IDF is an abbreviation for Term Frequency Inverse Document Frequency. It measures the relative importance of words by comparing the no. of times a word appear in a sample with the number of samples with that word.

This time the vectors are made up of the TF-IDFs of the words in the headings.

![](https://miro.medium.com/max/1400/1*qQgnyPLDIkUmeZKN2_ZWbQ.png)

In [13]:
## implement BAG OF WORDS
tfidfvector=TfidfVectorizer(ngram_range=(2,2))
traindataset=tfidfvector.fit_transform(headlines)

In [14]:
# implement RandomForest Classifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,train['Label'])

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [15]:
## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = tfidfvector.transform(test_transform)
predictions = randomclassifier.predict(test_dataset)

Evaluation for Random Forest + TF-IDF

In [16]:
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

[[149  37]
 [ 24 168]]
0.8386243386243386
              precision    recall  f1-score   support

           0       0.86      0.80      0.83       186
           1       0.82      0.88      0.85       192

    accuracy                           0.84       378
   macro avg       0.84      0.84      0.84       378
weighted avg       0.84      0.84      0.84       378



# Model - Naive Bayes
The accuracy is slightly lower than before. This may be due to the fact that common words (words which will appear in multiple documents) are actually helpful in distinguishing between classes in this case.

Next, we are going to use Naive Bayes Classifier.

Naive Bayes is another algorithm based on the Bayes Theorem for calculating probabilities and conditional probabilities. It can be extremely fast relative to other classification algorithms.

![](https://miro.medium.com/max/1200/1*39U1Ln3tSdFqsfQy6ndxOA.png)
For more detail on NB Classifier, check out the tutorial below:
https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/

In [17]:
naive=MultinomialNB()
naive.fit(traindataset,train['Label'])

MultinomialNB()

In [18]:
## Predict for the Test Dataset
test_transform= []
for row in range(0,len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row,2:27]))
test_dataset = tfidfvector.transform(test_transform)
predictions = naive.predict(test_dataset)

# Evaluation for NB + TF-IDF

In [19]:
matrix=confusion_matrix(test['Label'],predictions)
print(matrix)
score=accuracy_score(test['Label'],predictions)
print(score)
report=classification_report(test['Label'],predictions)
print(report)

[[130  56]
 [  0 192]]
0.8518518518518519
              precision    recall  f1-score   support

           0       1.00      0.70      0.82       186
           1       0.77      1.00      0.87       192

    accuracy                           0.85       378
   macro avg       0.89      0.85      0.85       378
weighted avg       0.89      0.85      0.85       378



# Results

This time we get a better result with 0 false negative. This shows that although Naive Bayes is generally considered the "quick fix", it may produce **equal or even better results** in many cases, especially in **textual applications**.

These results also showed that we can produce **decent results on NLP even without the use of Deep Learning** techniques, which is very encouraging for a new Data Scientist.

# Next Steps

1. It will be intersting to apply LSTM and see if deep learning can produce better result than Naive Bayes.
2. Writing an User Interface to make the project end-to-end.

In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import gensim.downloader as api

# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(headlines)
sequences = tokenizer.texts_to_sequences(headlines)
word_index = tokenizer.word_index
MAX_SEQUENCE_LENGTH = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Sample target labels (replace with your actual labels)
y_train = train['Label'].values

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, y_train, test_size=0.2, random_state=42)

# Load pre-trained Word2Vec embeddings
word2vec = api.load("word2vec-google-news-300")


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [21]:
embedding_dim = word2vec.vector_size
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))

for word, i in word_index.items():
    if word in word2vec:
        embedding_matrix[i] = word2vec[word]

def stack_model(layer_name, epochs=50):
    SEED = 42
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

    model = Sequential()
    model.add(Embedding(input_dim=len(word_index) + 1,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=False))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer="adam", metrics=['accuracy'])
    model.summary()

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, callbacks=[early_stopping])
    return model

# Example usage with LSTM layers
model = stack_model(LSTM, epochs=50)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 750, 300)          9721200   
                                                                 
 lstm (LSTM)                 (None, 750, 128)          219648    
                                                                 
 dropout (Dropout)           (None, 750, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 9990386 (38.11 MB)
Trainable params: 26918

In [24]:
# Predict for the test dataset
test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))

test_sequences = tokenizer.texts_to_sequences(test_transform)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

# Predict for the test dataset
predictions = model.predict(test_data)

predicted_labels = np.argmax(predictions, axis=1)
matrix = confusion_matrix(test['Label'], predicted_labels)
print("Confusion Matrix:\n", matrix)
score = accuracy_score(test['Label'], predicted_labels)
print("Accuracy Score:", score)
report = classification_report(test['Label'], predicted_labels)
print("Classification Report:\n", report)


12/12 [==============================] - 0s 21ms/step
Confusion Matrix:
 [[151  35]
 [ 69 123]]
Accuracy Score: 0.7248677248677249
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.81      0.74       186
           1       0.78      0.64      0.70       192

    accuracy                           0.72       378
   macro avg       0.73      0.73      0.72       378
weighted avg       0.73      0.72      0.72       378



In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report


vectorizer = TfidfVectorizer(ngram_range=(1, 2))
traindataset = vectorizer.fit_transform(headlines)

# Convert the sparse matrix to a dense format
X_train = traindataset.toarray()

# Sample target labels (replace with your actual labels)
y_train = train['Label'].values

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Step 3: Define the model
def stack_model(layer_name, epochs=50):
    SEED = 42
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

    INPUT_DIM = X_train.shape[1]  # Number of features (ngrams)
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(INPUT_DIM,)))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer="adam", metrics=['accuracy'])
    model.summary()

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, callbacks=[early_stopping])
    return model

# Example usage with Dense layers
model = stack_model(Dense, epochs=50)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 512)               223413760 
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               32896     
                                                                 
 dense_4 (Dense)             (None, 2)                 258       
                                                                 
Total params: 223578242 (852.88 MB)
Trainable params: 

In [26]:
# Predict for the test dataset
test_transform = []
for row in range(0, len(test.index)):
    test_transform.append(' '.join(str(x) for x in test.iloc[row, 2:27]))
test_dataset = vectorizer.transform(test_transform)

# Sort the indices of the sparse matrix
test_dataset.sort_indices()  # Add this line to sort indices

predictions = model.predict(test_dataset)

predicted_labels = np.argmax(predictions, axis=1)
matrix = confusion_matrix(test['Label'], predicted_labels)
print("Confusion Matrix:\n", matrix)
score = accuracy_score(test['Label'], predicted_labels)
print("Accuracy Score:", score)
report = classification_report(test['Label'], predicted_labels)
print("Classification Report:\n", report)

12/12 [==============================] - 0s 3ms/step
Confusion Matrix:
 [[152  34]
 [ 52 140]]
Accuracy Score: 0.7724867724867724
Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.82      0.78       186
           1       0.80      0.73      0.77       192

    accuracy                           0.77       378
   macro avg       0.77      0.77      0.77       378
weighted avg       0.78      0.77      0.77       378

